In [10]:
import mga.dataset as dataset
from pathlib import Path
import pandas as pd

In [11]:
dataset.download_ml_small(force_download=True)

'ratings.csv'

In [2]:
dataset.download_ml_25m()

'ratings.csv'

In [12]:
movie_df = pd.read_csv('movies.csv')
link_df = pd.read_csv('links.csv')

In [13]:
movie_df['imdb_id'] = link_df['imdbId']
movie_df['tmdb_id'] = link_df['tmdbId']
movie_df = movie_df.rename(columns={'movieId': 'movielens_id'})
movie_df[movie_df['genres'] == 'IMAX']['genres'] = '(no genres listed)'
movie_df.to_csv('movies.csv', index_label='id')

In [14]:
rating_df = pd.read_csv('ratings.csv')
del rating_df['timestamp']

movilens2id = pd.DataFrame(data=movie_df.index, index=movie_df['movielens_id'])
rating_df['movieId'] = movilens2id.loc[rating_df['movieId']].values

id2movielens = pd.Series(rating_df['userId'].drop_duplicates().sort_values().values)
movielens2id = pd.Series(data=id2movielens.index, index=id2movielens)
rating_df['userId'] = movielens2id.loc[rating_df['userId']].values

rating_df = rating_df.rename(columns={'userId': 'user_id'})
rating_df = rating_df.rename(columns={'movieId': 'movie_id'})
rating_df.to_csv('ratings.csv', index=False)